In [1]:
# import Library
import pandas as pd

In [2]:
# read source csv
# rename the downloaded csv as per your ease, Here we have renamed downloaded csv as build
df = pd.read_csv("build.csv")

# rename description column
df['Details'] = df['Accessibility type description']

# drop unwanted columns
df = df.drop(['CLUE small area', 'Construction year','Refurbished year', 'x coordinate','y coordinate','Accessibility type'
              ,'Has showers', 'Number of floors (above ground)', 'Bicycle spaces','Building name','Accessibility type description'], axis=1)

# Sort data by Census year, since we need the latest year data
df = df.sort_values(by=['Census year','Block ID', 'Property ID', 'Base property ID'])

In [3]:
# keeping the first observation because the first 1 reflects the csv observations which i believe are more accurate
df.drop(df[df.duplicated(['Block ID', 'Base property ID'], keep='last')== True].index, inplace=True)

# resetting the index
df = df.reset_index()
df.drop(['index'], inplace=True, axis=1)

In [4]:
# remove the brackets from the location variable
df['Location'] = df['Location'].map(lambda x: str(x)[:-1])
df['Location'] = df['Location'].map(lambda x: str(x)[1:])

In [5]:
# Derive latitude from location and convert it to numeric
df['Latitude'] = df['Location'].str.split(',').str[0]
df = df[df['Latitude'] != "a"]
df['Latitude'] = df['Latitude'].astype('float64')

In [6]:
# Derive Longitude from location and convert it to numeric
df['Longitude'] = df['Location'].str.split(',').str[1]
df['Longitude'] = df['Longitude'].astype('float64')

In [7]:
# drop unwanted columns
df = df.drop(['Block ID', 'Property ID', 'Base property ID','Location','Census year'], axis=1)

In [8]:
# drop unwanted records
df = df[df['Predominant space use'] != "House/Townhouse"]
df = df[df['Predominant space use'] != "Unoccupied - Unused"]
df = df[df['Predominant space use'] != "Unoccupied - Under Renovation"]
df = df[df['Predominant space use'] != "Unoccupied - Under Construction"]
df = df[df['Predominant space use'] != "Unoccupied - Under Demolition/Condemned"]
df = df[df['Predominant space use'] != "Hospital/Clinic"]
df = df[df['Predominant space use'] != "Manufacturing"]
df = df[df['Predominant space use'] != "Storage"]
df = df[df['Predominant space use'] != "Parking - Private Covered"]
df = df[df['Predominant space use'] != "Parking - Commercial Covered"]

In [9]:
# Derive accessibility/ disabled flag
df['Accessibility rating'] = df['Accessibility rating'].astype('str')

df = df.rename(index=str, columns={'Street address':'SA','Predominant space use':'ttype','Accessibility rating':'DisableFlag'})

df['DisableFlag'] =df['DisableFlag'].str.replace('0.0','No')
df['DisableFlag'] =df['DisableFlag'].str.replace('1.0','No')
df['DisableFlag'] =df['DisableFlag'].str.replace('2.0','Yes')
df['DisableFlag'] =df['DisableFlag'].str.replace('3.0','Yes')
df = df[df['DisableFlag'] != "nan"]
df = df.fillna("Blank_tobereplaced")

In [10]:
# derive primary key for the json file
df = df.reset_index()
df['PrimaryKey'] = df.index + 1
df['PrimaryKey'] = "B"+df.PrimaryKey.map(str)

df['Description'] = df.SA.map(str)
df['Category'] = df.ttype
df['Description'] =df['Description'].str.replace(' : Blank_tobereplaced','')

# clean the descriptions, handle blanks
df['Description1'] = df['Description'].str.rstrip('-').str.split('-').str[1]
df.fillna("Blank_tobereplaced", inplace=True)
for i in range(len(df)):  
    if df['Description1'][i] != "Blank_tobereplaced":  
        df['Description'][i] = df['Description1'][i]

C:\Users\RakeshOM\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [11]:
# drop unwanted columns
df = df.drop(['SA','ttype','index', 'Description1'], axis=1)

# reorder columns
df = df[['PrimaryKey', 'Category','Description', 'Latitude', 'Longitude', 'DisableFlag','Details']]

In [12]:
j = df.set_index('PrimaryKey').to_json(orient='index')

In [13]:
# save the dataframe to JSON file
with open("accessible_building.json", "w") as text_file:
    text_file.write("{}".format(j))

In [14]:
df

,PrimaryKey,Category,Description,Latitude,Longitude,DisableFlag,Details
0,B1,Entertainment/Recreation - Indoor,238 Little Bourke Street,-37.812229,144.965849,No,All entrances have steps
1,B2,Transport,201 La Trobe Street,-37.810191,144.962716,No,Configuration of entrance does not fit into an...
2,B3,Office,104 Cardigan Street,-37.804225,144.965141,No,All entrances have steps
3,B4,Residential Apartment,5 Rush Place,-37.804088,144.965716,No,All entrances have steps
4,B5,Office,50 Barry Street,-37.802930,144.960122,Yes,Main Entrance has ramp
5,B6,Residential Apartment,26 Little Curzon Street,-37.804160,144.948554,Yes,Main Entrance is at grade and has no steps or ...
6,B7,Residential Apartment,118 Peel Street,-37.805313,144.956442,No,All entrances have steps
7,B8,Residential Apartment,116 Peel Street,-37.805358,144.956432,No,All entrances have steps
8,B9,Office,24 Vale Street,-37.799731,144.953632,Yes,Entrance(s) have limited access via a small li...
9,B10,Student Accommodation,6 High Street,-37.799825,144.953990,No,All entrances have steps
